In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays # 可选，用于处理稀疏性（如果数据量很大）

include("Data_Generation.jl")
include("Estimation_Wang_Qi_Max.jl")

estimate_parameters (generic function with 1 method)

In [2]:
d = 5 # num of product feature
p = 10 # num of customer feature
n = 1000 # num of training samples
n_test = 1000
m = 5 # num of products
s = 10
Random.seed!(2024)

TaskLocalRNG()

### Data Generation

In [3]:
theta_true, r_params = Generate_Wang_Qi_Max_True_Paras(d,p,s);

In [4]:
X_train,Y_train,Z_train = Generate_Wang_Qi_Max_True_Data(d, p, n, m,theta_true);

In [5]:
X_test,Y_test,Z_test = Generate_Wang_Qi_Max_True_Data(d, p, n_test, m,theta_true);

### Estimation

In [ ]:
lambda = 0.0001
theta_hat, opt_result = estimate_parameters(X_train,Y_train,Z_train,lambda, d, p, initial_theta=randn((d+1)*(p+1)) * 0.1);

### Estimate-then-optimize model

In [127]:
function Product_Design_Ours_ETO(d,nu0, nu, r0, r, z_input)

    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    # 定义变量
    @variable(model, a0 >= 0)                       
    @variable(model, a1 >= 0) 
    @variable(model, u)                       
    @variable(model, v)
    @variable(model, x[1:d], Bin)
    @variable(model, y[1:d])
    @variable(model, z_int[1:d])

    @constraint(model, sum(x) >= 1)
    @constraint(model, a0 + a1 == 1)
    @constraint(model, [u,a0,a1] in MOI.ExponentialCone())
    @constraint(model, [-v,a1,a0] in MOI.ExponentialCone())
    @constraint(model, u == nu0 * a0 + nu' * z_int)
    @constraint(model, v == a1 * nu0 + nu' * y)
    for ind in 1:d
        @constraint(model, z_int[ind] <= a0)
        @constraint(model, z_int[ind] <= x[ind])
        @constraint(model, z_int[ind] >= x[ind] - (1 - a0))
        @constraint(model, z_int[ind] >= 0)
    end
    for ind in 1:d
        @constraint(model, y[ind] <= a1)
        @constraint(model, y[ind] <= x[ind])
        @constraint(model, y[ind] >= x[ind] - (1 - a1))
        @constraint(model, y[ind] >= 0)
    end
    @objective(model, Max,r0 * a1 + r' * y)
    optimize!(model)
    status = JuMP.termination_status(model)
    if status == MOI.OPTIMAL
        obj_val = objective_value(model)
        x_val = value.(x)
        solve_time = JuMP.solve_time(model)
    else
        obj_val = NaN
        x_val = ones(d) .* NaN
        solve_time = NaN
    end
    return obj_val, x_val, solve_time
end

Product_Design_Ours_ETO (generic function with 1 method)

In [137]:
function Product_Design_ETO(d,nu0, nu, r0,r, z_input)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    # 定义变量
    @variable(model, a0 >= 0)                       
    @variable(model, a1 >= 0) 
    @variable(model, b0)                       
    @variable(model, b1) 
    @variable(model, v0)                       
    @variable(model, v1)
    @variable(model, u)                       
    @variable(model, w)
    @variable(model, x[1:d], Bin)
    @variable(model, y[1:d])
    @variable(model, phi)

    @constraint(model, a0 + a1 == 1)
    @constraint(model, w + b1 + b0 >= phi)
    @constraint(model, [b1,a1,1] in MOI.ExponentialCone())
    @constraint(model, [b0,a0,1] in MOI.ExponentialCone())
    @constraint(model, v0 + v1 <= 1)
    @constraint(model, [u-phi,1,v1] in MOI.ExponentialCone())
    @constraint(model, [-phi,1,v0] in MOI.ExponentialCone())
    @constraint(model, w == nu0 * a1 + nu' * y)
    @constraint(model, u == nu0 + nu' * x)
    for ind in 1:d
        @constraint(model, y[ind] <= a1)
        @constraint(model, y[ind] <= x[ind])
        @constraint(model, y[ind] >= x[ind] - (1 - a1))
        @constraint(model, y[ind] >= 0)
    end
    @objective(model, Max,r0 * a1 + r' * y)

    optimize!(model)
    status = JuMP.termination_status(model)

    if status == MOI.OPTIMAL
        obj_val = objective_value(model)
        x_val = value.(x)
        solve_time = JuMP.solve_time(model)
    else
        obj_val = NaN
        x_val = ones(d) .* NaN
        solve_time = NaN
    end
    return obj_val, x_val, solve_time
end

Product_Design_ETO (generic function with 2 methods)

In [154]:
z_input = Z_test[3,:];
r0 = r_params.r0;
r = r_params.r;
alp0 = theta_true.alpha0;
alp = theta_true.alpha;
beta = theta_true.beta;
A = theta_true.A;
nu0 = alp0 + beta' * z_input;
nu = alp .+ A * z_input;

In [155]:
obj_True_Our, x_True_our, solve_time_true = Product_Design_Ours_ETO(d,nu0, nu, r0, r, z_input);

In [156]:
obj_True, x_True, solve_time = Product_Design_ETO(d,nu0, nu, r0, r, z_input);

In [159]:
println("objective value of our model: ", obj_True_Our)
println("objective value of Wang_Qi_Max (2019): ", obj_True)

objective value of our model: 0.7871483882747676
objective value of Wang_Qi_Max (2019): 0.7873610960644186


### Performance

In [161]:
function calculate_profit(alp0, alp, beta, A, r0, r, x_val, z_input)
    utility = alp0 + alp' * x_val + beta' * z_input + x_val' * A * z_input
    prob = exp(utility)/(1+exp(utility))
    profits = r0 + r' * x_val
    total_profit = profits * prob
    return total_profit
end

calculate_profit (generic function with 1 method)

In [162]:
profit_True_ours = calculate_profit(alp0, alp, beta, A, r0, r, x_True_our, z_input)
profit_True = calculate_profit(alp0, alp, beta, A, r0, r, x_True, z_input)
println("total profit of our model: ", profit_True_ours)
println("total profit of Wang_Qi_Max (2019): ", profit_True)

total profit of our model: 0.7871483877515713
total profit of Wang_Qi_Max (2019): 0.7871483877515713
